In [ ]:
# Image mapper demonstrator
# Author: Tjark Miener, Universidad Complutense de Madrid (tmiener@ucm.es)

from ctlearn.image_mapping import ImageMapper
import numpy as np
import matplotlib.pyplot as plt
from tables import *
from astropy.io import fits
%matplotlib inline

In [ ]:
# plotting function 
def plot_image(mapper, camera_type):
    print(camera_type)
    image = mapper.map_image(test_pixel_values[camera_type], camera_type)
    fig, ax = plt.subplots(1)
    ax.set_aspect(1)
    ax.pcolor(image[:,:,0], cmap='viridis')
    plt.show()

In [ ]:
# Load the image mappers
cam_types = ['FlashCam','LSTCam']
hex_algos = ['oversampling', 'rebinning', 'nearest_interpolation',
             'bilinear_interpolation', 'bicubic_interpolation', 
             'image_shifting', 'axial_addressing']

mappers = {}
for hex_algo in hex_algos:
    print(hex_algo)
    hex_conversion_algorithm = {cam: hex_algo for cam in cam_types}
    %time mappers[hex_algo]=ImageMapper(camera_types=cam_types,hex_conversion_algorithm=hex_conversion_algorithm)

In [ ]:
#  pixel values (value is the pixel number)
f = open_file('../data/dl1-data-handler/proton_20deg_0deg_runs100-103___cta-prod3-demo_desert-2150m-Paranal-baseline-sample.h5', 'r')
LST_images = f.root.LST_LSTCam.col('charge')
Flash_images = f.root.MST_FlashCam.col('charge')

image_charge_values = {}
image_charge_dict = {}
LST_image_index = 9
Flash_image_index = 77
for cam in cam_types:
        if cam == 'LSTCam':
            image_charge = LST_images[LST_image_index]
            file = "./pixel_pos_files/LSTCam.camgeom.fits"
        elif cam == 'FlashCam':
            image_charge = Flash_images[Flash_image_index]
            file = "./pixel_pos_files/FlashCam.camgeom.fits"
            
        # read the pixel positions from the fits file
        hdul = fits.open(file)
        data = hdul[1].data
        header = hdul[1].header
        x = data.field('pix_x')
        y = data.field('pix_y')
        pos = np.column_stack([x,y]).T

        # set up the image charge in the right form to pass to the map_image function
        image_charge_values[cam] = np.concatenate(([0.0], image_charge))
        image_charge_values[cam] = np.expand_dims(image_charge_values[cam], axis=1)
        
        # plot the image charge in a hexagonal lattice (this is what the camera sees)
        print("{} - hexagonal".format(cam))
        plt.figure()
        plt.scatter(pos[0],pos[1],c=image_charge,s=15,marker='H')
        plt.axes().set_aspect('equal')
        plt.show()
        plt.close()
        
        for hex_algo in hex_algos:
            print("{} - {}".format(cam, hex_algo))
            image_charge_dict[cam] = mappers[hex_algo].map_image(image_charge_values[cam], cam)
            # Create vectorial grapics for each telescope camera with oversampling.
            plt.figure()
            plt.pcolor(image_charge_dict[cam][:,:,0],cmap='viridis')
            plt.axes().set_aspect('equal')
            plt.show()
            plt.close()